## Installing and Importing

In [1]:
PROJECT_ID = "qwiklabs-gcp-01-c1e65f3fd08c"
NETWORK_NAME = "default"
PEERING_RANGE_NAME = "cymbal-range"

# Reserve IP range
! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for cymbal demo"

# Set up peering with service networking
! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={NETWORK_NAME} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

Created [https://www.googleapis.com/compute/v1/projects/qwiklabs-gcp-01-c1e65f3fd08c/global/addresses/cymbal-range].
Operation "operations/pssn.p24-738058776700-81a3b24e-ba72-4652-8dcf-bdf6e62afe0f" finished successfully.


In [2]:
! pip install -U git+https://github.com/googleapis/python-aiplatform.git@main --user

  Cloning https://github.com/googleapis/python-aiplatform.git (to revision main) to /tmp/pip-req-build-a36n8p9v
  Running command git clone --filter=blob:none --quiet https://github.com/googleapis/python-aiplatform.git /tmp/pip-req-build-a36n8p9v
  Resolved https://github.com/googleapis/python-aiplatform.git to commit 22b04fcaa353ead4b1e4441ff3eb3a96257f6b00
  Preparing metadata (setup.py) ... done
  Created wheel for google-cloud-aiplatform: filename=google_cloud_aiplatform-1.57.0-py2.py3-none-any.whl size=5122065 sha256=c689cc4b51cbd8bef544a3194190484ba6d6722f60d4aa790caa4ea3da81ce56
  Stored in directory: /tmp/pip-ephem-wheel-cache-wsw3d301/wheels/37/8f/6d/f08d62dc5b559f664c7a756630645dbf5ca612fe1b294ccf8c
Successfully built google-cloud-aiplatform
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
! pip install -U grpcio-tools --user
! pip install -U h5py --user

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.57.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-artifact-registry 1.11.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-connection 1.15.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0

In [4]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
BUCKET_NAME = "gs://qwiklabs-gcp-01-c1e65f3fd08c-aip"
REGION = "us-west1"
PROJECT_ID = "qwiklabs-gcp-01-c1e65f3fd08c"
NETWORK_NAME = "default"

In [2]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_NAME

Creating gs://qwiklabs-gcp-01-c1e65f3fd08c-aip/...


In [3]:
! gsutil ls -al $BUCKET_NAME

In [6]:
! pip install proto-plus==1.24.0.dev1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.2 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.2
    Uninstalling protobuf-5.27.2:
      Successfully uninstalled protobuf-5.27.2
  Attempting uninstall: proto-plus
    Found existing installation: proto-plus 1.23.0
    Uninstalling proto-plus-1.23.0:
      Successfully uninstalled proto-plus-1.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.64.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.3 which is incompatible.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.3 which is incompatible.
google-cloud-datastore 1.15.5 re

In [7]:
import time

import grpc
import h5py
from google.cloud import aiplatform_v1
from google.protobuf import struct_pb2

In [8]:
REGION = "us-west1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
NETWORK_NAME = "default"


AUTH_TOKEN = !gcloud auth print-access-token
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

print("ENDPOINT: {}".format(ENDPOINT))
print("PROJECT_ID: {}".format(PROJECT_ID))
print("REGION: {}".format(REGION))

!gcloud config set project {PROJECT_ID} --quiet
!gcloud config set ai_platform/region {REGION} --quiet

ENDPOINT: us-west1-aiplatform.googleapis.com
PROJECT_ID: qwiklabs-gcp-01-c1e65f3fd08c
REGION: us-west1
Updated property [core/project].
Updated property [ai_platform/region].


## Downloading Data

In [9]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5 .

Copying gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5...
/ [1 files][462.9 MiB/462.9 MiB]                                                
Operation completed over 1 objects/462.9 MiB.                                    


In [10]:
# The number of nearest neighbors to be retrieved from database for each query.
k = 10

h5 = h5py.File("glove-100-angular.hdf5", "r")
train = h5["train"]
test = h5["test"]

In [11]:
train[0]

array([-0.11333  ,  0.48402  ,  0.090771 , -0.22439  ,  0.034206 ,
       -0.55831  ,  0.041849 , -0.53573  ,  0.18809  , -0.58722  ,
        0.015313 , -0.014555 ,  0.80842  , -0.038519 ,  0.75348  ,
        0.70502  , -0.17863  ,  0.3222   ,  0.67575  ,  0.67198  ,
        0.26044  ,  0.4187   , -0.34122  ,  0.2286   , -0.53529  ,
        1.2582   , -0.091543 ,  0.19716  , -0.037454 , -0.3336   ,
        0.31399  ,  0.36488  ,  0.71263  ,  0.1307   , -0.24654  ,
       -0.52445  , -0.036091 ,  0.55068  ,  0.10017  ,  0.48095  ,
        0.71104  , -0.053462 ,  0.22325  ,  0.30917  , -0.39926  ,
        0.036634 , -0.35431  , -0.42795  ,  0.46444  ,  0.25586  ,
        0.68257  , -0.20821  ,  0.38433  ,  0.055773 , -0.2539   ,
       -0.20804  ,  0.52522  , -0.11399  , -0.3253   , -0.44104  ,
        0.17528  ,  0.62255  ,  0.50237  , -0.7607   , -0.071786 ,
        0.0080131, -0.13286  ,  0.50097  ,  0.18824  , -0.54722  ,
       -0.42664  ,  0.4292   ,  0.14877  , -0.0072514, -0.1648

In [12]:
# Add restricts to each data point, in this demo, we only add one namespace and the allowlist is set to be the same as the id.
# Later on, we will demo how to return only the allowlisted data points.
# Split datapoins into two groups 'a' and 'b'. The datapoint whose ids are even are in group 'a', otherwise are in group 'b'
# We will demo how to configure the query to return up to k data points for each group.
with open("glove100.json", "w") as f:
    for i in range(len(train)):
        f.write('{"id":"' + str(i) + '",')
        f.write('"embedding":[' + ",".join(str(x) for x in train[i]) + "],")
        f.write('"restricts":[{"namespace": "class", "allow": ["' + str(i) + '"]}],')
        f.write('"crowding_tag":' + ('"a"' if i % 2 == 0 else '"b"') + "}")
        f.write("\n")
        if i >= 100:
            break

In [13]:
# NOTE: Everything in this GCS DIR will be DELETED before uploading the data.

! gsutil rm -rf {BUCKET_NAME}/*

CommandException: 1 files/objects could not be removed.


In [14]:
! gsutil cp glove100.json {BUCKET_NAME}/glove100.json

Copying file://glove100.json [Content-Type=application/json]...
/ [1 files][ 93.1 KiB/ 93.1 KiB]                                                
Operation completed over 1 objects/93.1 KiB.                                     


In [15]:
! gsutil ls {BUCKET_NAME}

gs://qwiklabs-gcp-01-c1e65f3fd08c-aip/glove100.json


##  Create Stream Update Index

In [16]:
index_client = aiplatform_v1.IndexServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [17]:
DIMENSIONS = 100
DISPLAY_NAME = "glove_100_1"

In [18]:
treeAhConfig = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(number_value=500),
        "leafNodesToSearchPercent": struct_pb2.Value(number_value=7),
    }
)

algorithmConfig = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=150),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=BUCKET_NAME),
    }
)

ann_index = {
    "display_name": DISPLAY_NAME,
    "description": "Glove 100 ANN index",
    "metadata": struct_pb2.Value(struct_value=metadata),
    "index_update_method": aiplatform_v1.Index.IndexUpdateMethod.STREAM_UPDATE,
}

In [19]:
ann_index = index_client.create_index(parent=PARENT, index=ann_index)

In [20]:
ann_index.result()

name: "projects/738058776700/locations/us-west1/indexes/6421156702304862208"
display_name: "glove_100_1"
description: "Glove 100 ANN index"
metadata_schema_uri: "gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml"
metadata {
  struct_value {
    fields {
      key: "config"
      value {
        struct_value {
          fields {
            key: "shardSize"
            value {
              string_value: "SHARD_SIZE_MEDIUM"
            }
          }
          fields {
            key: "distanceMeasureType"
            value {
              string_value: "DOT_PRODUCT_DISTANCE"
            }
          }
          fields {
            key: "dimensions"
            value {
              number_value: 100
            }
          }
          fields {
            key: "approximateNeighborsCount"
            value {
              number_value: 150
            }
          }
          fields {
            key: "algorithmConfig"
            value {
    

In [21]:
INDEX_RESOURCE_NAME = ann_index.result().name
INDEX_RESOURCE_NAME

'projects/738058776700/locations/us-west1/indexes/6421156702304862208'

## Create an IndexEndpoint with VPC Network

In [22]:
index_endpoint_client = aiplatform_v1.IndexEndpointServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [23]:
VPC_NETWORK_NAME = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, NETWORK_NAME)
VPC_NETWORK_NAME

'projects/738058776700/global/networks/default'

In [24]:
index_endpoint = {
    "display_name": "index_endpoint_for_demo",
    "network": VPC_NETWORK_NAME,
}

In [25]:
r = index_endpoint_client.create_index_endpoint(
    parent=PARENT, index_endpoint=index_endpoint
)

In [26]:
r.result()

name: "projects/738058776700/locations/us-west1/indexEndpoints/3898050195442630656"
display_name: "index_endpoint_for_demo"
network: "projects/738058776700/global/networks/default"
encryption_spec {
}

In [27]:
INDEX_ENDPOINT_NAME = r.result().name
INDEX_ENDPOINT_NAME

'projects/738058776700/locations/us-west1/indexEndpoints/3898050195442630656'

## Deploy Stream Update Index

In [28]:
DEPLOYED_INDEX_ID = "stream_update_glove_deployed"

In [29]:
deploy_ann_index = {
    "id": DEPLOYED_INDEX_ID,
    "display_name": DEPLOYED_INDEX_ID,
    "index": INDEX_RESOURCE_NAME,
}

In [30]:
r = index_endpoint_client.deploy_index(
    index_endpoint=INDEX_ENDPOINT_NAME, deployed_index=deploy_ann_index
)

In [31]:
# Poll the operation until it's done successfullly.

while True:
    if r.done():
        break
    print("Poll the operation to deploy index...")
    time.sleep(60)
     

Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...


KeyboardInterrupt: 

In [ ]:
r.result()